In [1]:
# build_crime_features.py

In [2]:
# build_crime_long_full.py
import pandas as pd
from pathlib import Path

LOOKUP_CSV = Path("london_lsoa_lookup.csv")

# 1 | London LSOA list
london = pd.read_csv(LOOKUP_CSV, dtype=str)["LSOA_Code"]

# 2 | read & concat wide CSVs
dtype_map = {
    "LSOA Code": "string",
    "LSOA Name": "string",
    "Borough": "string",
    "Major Category": "category",
    "Minor Category": "category",
}

wide_files = [
    Path("MPS LSOA Level Crime (Historical).csv"),
    Path("MPS LSOA Level Crime (most recent 24 months).csv"),
]

dfs = []
for fp in wide_files:
    df = pd.read_csv(fp, dtype=dtype_map, low_memory=False)
    dfs.append(df)
wide = pd.concat(dfs, ignore_index=True)

wide = wide.drop_duplicates(
    subset=["LSOA Code", *[c for c in wide.columns if c.isdigit() and len(c)==6]],
    keep="first"
)

wide.columns = (wide.columns
                 .str.strip()
                 .str.replace(" ", "_")
                 .str.replace("LSOA_Code", "LSOA_Code", regex=False))
id_cols    = ["LSOA_Code","LSOA_Name","Borough","Major_Category","Minor_Category"]
month_cols = [c for c in wide.columns if c.isdigit() and len(c) == 6]

crime_long = (
    wide.melt(id_vars=id_cols,
              value_vars=month_cols,
              var_name="yyyymm",
              value_name="crime_cnt")
        .assign(
            crime_cnt=lambda df: df.crime_cnt.fillna(0).astype("int32"),
            date=lambda df: pd.to_datetime(df.yyyymm + "01", format="%Y%m%d")
        )
        .query("LSOA_Code in @london")
        .reset_index(drop=True)
)

# 3 | save
crime_long.to_csv("crime_long_london.csv", index=False)
print("✓ crime_long_london.csv saved:", crime_long.shape)

✓ crime_long_london.csv saved: (36364113, 8)


In [3]:
import pandas as pd


usecols = ["LSOA_Code", "date"]


row_count = sum(1 for _ in open("crime_long_london.csv")) - 1   # minus header
print("rows in file:", f"{row_count:,}")


dates_min, dates_max = None, None
lsoa_set = set()

for chunk in pd.read_csv("crime_long_london.csv", usecols=usecols,
                         parse_dates=["date"], chunksize=500_000):
    lsoa_set.update(chunk["LSOA_Code"].unique())
    dmin, dmax = chunk["date"].min(), chunk["date"].max()
    dates_min = dmin if dates_min is None or dmin < dates_min else dates_min
    dates_max = dmax if dates_max is None or dmax > dates_max else dates_max

print("unique LSOA:", len(lsoa_set))
print("date range :", dates_min, "→", dates_max)

rows in file: 36,364,113
unique LSOA: 4653
date range : 2010-04-01 00:00:00 → 2025-06-01 00:00:00


In [4]:
# crime_type_features.py
import pandas as pd

# 1 | load long-form crime table
crime = pd.read_csv("crime_long_london.csv",
                    usecols=["LSOA_Code","date","Major_Category","crime_cnt"],
                    parse_dates=["date"])

# 2 | pivot: one column per Major_Category
crime_pivot = (
    crime.pivot_table(index=["LSOA_Code","date"],
                      columns="Major_Category",
                      values="crime_cnt",
                      aggfunc="sum",
                      fill_value=0)
          .reset_index()
)

crime_pivot.columns = (
    ["LSOA_Code","date"] +
    [f"cnt_{c.replace(' ','_')}" for c in crime_pivot.columns[2:]]
)

crime_pivot.to_csv("crime_monthly_wide.csv", index=False)
print("✓ monthly pivot saved → crime_monthly_wide.csv  ", crime_pivot.shape)

# 3 | rolling 6‑month sum
crime_pivot = crime_pivot.sort_values(["LSOA_Code","date"])

category_cols = [c for c in crime_pivot.columns if c.startswith("cnt_")]

for col in category_cols:
    crime_pivot[f"{col}_roll6"] = (
        crime_pivot
        .groupby("LSOA_Code")[col]
        .rolling(window=6, min_periods=1).sum()    # recent 6 months
        .reset_index(level=0, drop=True)
    )


crime_roll6 = crime_pivot[["LSOA_Code","date"] +
                          [f"{c}_roll6" for c in category_cols]]

crime_roll6.to_csv("crime_rolling6m.csv", index=False)
print("✓ rolling‑6m features saved → crime_rolling6m.csv  ", crime_roll6.shape)

✓ monthly pivot saved → crime_monthly_wide.csv   (851499, 12)
✓ rolling‑6m features saved → crime_rolling6m.csv   (851499, 12)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 读取数据
crime_df = pd.read_csv("crime_monthly_wide.csv", parse_dates=["date"])

# 每月的唯一 LSOA 数量
monthly_coverage = crime_df.groupby("date")["LSOA_Code"].nunique().reset_index()
monthly_coverage.columns = ["date", "num_lsoas"]

# 绘图
plt.figure(figsize=(10, 5))
plt.plot(monthly_coverage["date"], monthly_coverage["num_lsoas"], linewidth=2)
plt.title("Monthly LSOA Coverage in Crime Data (2010.04 – 2025.06)")
plt.xlabel("Date")
plt.ylabel("Number of LSOAs")
plt.grid(True)
plt.tight_layout()
plt.savefig("crime_data_coverage.png", dpi=150)
plt.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np

# Set plot style
sns.set_style("whitegrid")
plt.rcParams['figure.facecolor'] = 'white'

# Read data

df = pd.read_csv('crime_monthly_wide.csv', nrows=1000)

# Data preprocessing
# Convert date column to datetime format
df['date'] = pd.to_datetime(df['date'])
# Extract year
df['year'] = df['date'].dt.year

# Define crime type column names (remove cnt_ prefix for better display)
crime_columns = [
    'cnt_ARSON_AND_CRIMINAL_DAMAGE',
    'cnt_BURGLARY', 
    'cnt_DRUG_OFFENCES',
    'cnt_MISCELLANEOUS_CRIMES_AGAINST_SOCIETY',
    'cnt_POSSESSION_OF_WEAPONS',
    'cnt_PUBLIC_ORDER_OFFENCES',
    'cnt_ROBBERY',
    'cnt_THEFT',
    'cnt_VEHICLE_OFFENCES',
    'cnt_VIOLENCE_AGAINST_THE_PERSON'
]

# Aggregate crime data by year
yearly_crime = df.groupby('year')[crime_columns].sum().reset_index()

# Create English labels for crime types
crime_labels = {
    'cnt_ARSON_AND_CRIMINAL_DAMAGE': 'Arson & Criminal Damage',
    'cnt_BURGLARY': 'Burglary',
    'cnt_DRUG_OFFENCES': 'Drug Offences',
    'cnt_MISCELLANEOUS_CRIMES_AGAINST_SOCIETY': 'Miscellaneous Crimes',
    'cnt_POSSESSION_OF_WEAPONS': 'Weapon Possession',
    'cnt_PUBLIC_ORDER_OFFENCES': 'Public Order Offences',
    'cnt_ROBBERY': 'Robbery',
    'cnt_THEFT': 'Theft',
    'cnt_VEHICLE_OFFENCES': 'Vehicle Offences',
    'cnt_VIOLENCE_AGAINST_THE_PERSON': 'Violence Against Person'
}

# All crime types in one plot
plt.figure(figsize=(15, 10))

for column in crime_columns:
    plt.plot(yearly_crime['year'], yearly_crime[column], 
             marker='o', linewidth=2, label=crime_labels[column])

plt.title('Annual Crime Trends by Type', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Number of Crimes', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
